In [1]:
%pip install numpy
%pip install opencv-python
%pip install scipy
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal

In [3]:
# declare prewitt kernel
_PREWITT_X_KERNEL = np.array([
    [-1, 0 ,1],
    [-1, 0, 1],
    [-1, 0, 1]
])

_PREWITT_Y_KERNEL = np.array([
    [-1, -1, -1],
    [0, 0, 0],
    [1, 1, 1]
])

In [16]:
def show_image(name):
    image_path = pathlib.Path(f'./assets/{name}.jpg')

    # image is in BGR
    image_matrix = cv2.imread(str(image_path))
    grayscale_image = cv2.cvtColor(image_matrix, cv2.COLOR_BGR2GRAY)


    cv2.imshow(f'{name}', image_matrix)
    cv2.imshow(f'grayscale-{name}', grayscale_image)

    # search the edge based on axis (the axis corresspond with kernel axis)
    convolved_image_x = cv2.filter2D(grayscale_image, -1, _PREWITT_X_KERNEL)
    convolved_image_y = cv2.filter2D(grayscale_image, -1, _PREWITT_Y_KERNEL)


    # cv2.imshow(f'convolved image (x) - {name}', convolved_image_x)
    # cv2.imshow(f'convolved image (y) - {name}', convolved_image_y)
    # cv2.imshow(f'convolved image - {name}', convolved_image_x + convolved_image_y)


    figure = plt.figure(figsize=(14, 7))
    
    figure.add_subplot(2, 5, 1)
    plt.imshow(plt.imread(image_path))
    plt.axis("off")
    plt.title("Kernel X")


    figure.add_subplot(2, 5, 2)
    plt.imshow(grayscale_image, cmap='grey')
    plt.axis("off")
    plt.title("Kernel X")

    figure.add_subplot(2, 5, 3)
    plt.imshow(convolved_image_x, cmap='grey')
    plt.axis("off")
    plt.title("Kernel X")

    figure.add_subplot(2, 5, 4)
    plt.imshow(convolved_image_y, cmap='grey')
    plt.axis("off")
    plt.title("Kernel Y")

    figure.add_subplot(2, 5, 5)
    plt.imshow(convolved_image_x + convolved_image_y, cmap='grey')
    plt.axis("off")
    plt.title("Kernel X + Y")

show_image('apel')
# show_image('knife_image')
# show_image('meja')
# show_image('kursi')


cv2.waitKey(0)
cv2.destroyAllWindows()

ValueError: num must be an integer with 1 <= num <= 10, not 0

<Figure size 1400x700 with 0 Axes>